## HOMEWORK - MODULE 5

In [38]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [39]:
spark.version

'3.5.5'

Read the October 2024 Yellow into a Spark Dataframe.

In [40]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

Repartition the Dataframe to 4 partitions and save it to parquet.

In [41]:
df = df.repartition(4)

Save it to parquet

In [ ]:
output_path="C:/Users/Sandra/Documents/GitHub/Data_Engineering_Zoomcamp_2025/5_batch/files"
df.write.parquet(output_path, mode="overwrite")

How many taxi trips were there on the 15th of October?

In [47]:
df.registerTempTable('trip')  

spark.sql("""
SELECT count(*) FROM trip 
WHERE DATE(tpep_pickup_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



What is the length of the longest trip in the dataset in hours?

In [54]:
spark.sql("""
SELECT 
    ROUND((unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600, 1) AS trip_duration
FROM trip
ORDER BY trip_duration DESC
LIMIT 1;
""").show()  

+-------------+
|trip_duration|
+-------------+
|        162.6|
+-------------+



Load the zone lookup data into a temp view in Spark:

> `wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv`

What is the name of the LEAST frequent pickup location Zone?

In [55]:
lookup_df = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

lookup_df.registerTempTable('lookup')  

spark.sql("""

SELECT lookup.Zone , count(*) as num_trips FROM trip 
INNER JOIN lookup ON lookup.LocationID = trip.PULocationID
GROUP BY lookup.Zone
ORDER BY num_trips ASC
LIMIT 1;
""").show() 

+--------------------+---------+
|                Zone|num_trips|
+--------------------+---------+
|Governor's Island...|        1|
+--------------------+---------+

